In [ ]:
import ee
import geemap
import time


In [ ]:
# ee.Authenticate()
ee.Initialize()

In [ ]:
roi = ee.Geometry.BBox(30, 30, 31.333, 30.99)

Map = geemap.Map(center=[30.5, 30.5], zoom=8)

grid_width = 0.25
grid_height = 0.25

start = "2025-01-01"
end = "2025-02-28"

composite_image = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterDate(start, end)
    .filterBounds(roi)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .median()
    .clip(roi)
    .select(['B4', 'B3', 'B2'])
)

def make_fishnet(region, dx, dy, image):
    coords = region.bounds().coordinates().get(0)
    coords = ee.List(coords)

    xmin = ee.Number(ee.List(coords.get(0)).get(0))
    ymin = ee.Number(ee.List(coords.get(0)).get(1))
    xmax = ee.Number(ee.List(coords.get(2)).get(0))
    ymax = ee.Number(ee.List(coords.get(2)).get(1))

    x_cells = xmax.subtract(xmin).divide(dx).ceil()
    y_cells = ymax.subtract(ymin).divide(dy).ceil()

    def create_row(j):
        j = ee.Number(j)
        y1 = ymin.add(j.multiply(dy))
        y2 = y1.add(dy)

        def create_cell(i):
            i = ee.Number(i)
            x1 = xmin.add(i.multiply(dx))
            x2 = x1.add(dx)
            rect = ee.Geometry.Rectangle([x1, y1, x2, y2])
            tile_img = image.clip(rect)

            file_name = ee.String('tile_') \
                .cat(x1.format('%.3f')).cat('_') \
                .cat(y1.format('%.3f')).cat('_') \
                .cat(x2.format('%.3f')).cat('_') \
                .cat(y2.format('%.3f'))

            return ee.Feature(rect, {'file_name': file_name})

        return ee.List.sequence(0, x_cells.subtract(1)).map(create_cell)

    grid = ee.List.sequence(0, y_cells.subtract(1)).map(lambda j: create_row(j)).flatten()
    return ee.FeatureCollection(grid)

fishnet = make_fishnet(roi, grid_width, grid_height, composite_image)

features = fishnet.getInfo()['features']

for feature in features:
    geom = ee.Geometry(feature['geometry'])
    file_name = feature['properties']['file_name']

    task = ee.batch.Export.image.toDrive(
        image=composite_image.clip(geom),
        description=file_name,
        folder='S2_tiles',
        fileNamePrefix=file_name,
        region=geom,
        scale=10,
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    task.start()
    print(f"Started task: {file_name}")
    time.sleep(1) 

Started task: tile_30.000_30.000_30.250_30.250
Started task: tile_30.250_30.000_30.500_30.250
Started task: tile_30.500_30.000_30.750_30.250
Started task: tile_30.750_30.000_31.000_30.250
Started task: tile_31.000_30.000_31.250_30.250
Started task: tile_31.250_30.000_31.500_30.250
Started task: tile_30.000_30.250_30.250_30.500
Started task: tile_30.250_30.250_30.500_30.500
Started task: tile_30.500_30.250_30.750_30.500
Started task: tile_30.750_30.250_31.000_30.500
Started task: tile_31.000_30.250_31.250_30.500
Started task: tile_31.250_30.250_31.500_30.500
Started task: tile_30.000_30.500_30.250_30.750
Started task: tile_30.250_30.500_30.500_30.750
Started task: tile_30.500_30.500_30.750_30.750
Started task: tile_30.750_30.500_31.000_30.750
Started task: tile_31.000_30.500_31.250_30.750
Started task: tile_31.250_30.500_31.500_30.750
Started task: tile_30.000_30.750_30.250_31.000
Started task: tile_30.250_30.750_30.500_31.000
Started task: tile_30.500_30.750_30.750_31.000
Started task:

In [ ]:
Map.addLayer(fishnet.style(**{'color': 'blue', 'fillColor': '00000000'}), {}, "Fishnet Grid")
Map.addLayer(roi, {"color": "red"}, "ROI")
Map.addLayer(
    composite_image,
    {
        'bands': ['B4', 'B3', 'B2'],  # RGB
        'min': 0,
        'max': 3000,
        'gamma': 1.3 
    },
    'S2 Composite (2025)'
) 
Map

Map(center=[30.5, 30.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…